In [1]:
#This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
# -*- coding: utf-8 -*-
"""
Created on Wed Oct  4 21:39:52 2017

@author: bhavesh
"""
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
import xgboost as xgb

cwd=os.getcwd()+'/'

def gini(actual, pred, cmpcol = 0, sortcol = 1):
    assert( len(actual) == len(pred) )
    all = np.asarray(np.c_[ actual, pred, np.arange(len(actual)) ], dtype=np.float)
    all = all[ np.lexsort((all[:,2], -1*all[:,1])) ]
    totalLosses = all[:,0].sum()
    giniSum = all[:,0].cumsum().sum() / totalLosses
    
    giniSum -= (len(actual) + 1) / 2.
    return giniSum / len(actual)

def gini_normalized(a, p):
    return gini(a, p) / gini(a, a)

def gini_xgb(preds, dtrain):
    labels = dtrain.get_label()
    gini_score = gini_normalized(labels, preds)
    return [('gini', gini_score)]

df_train = pd.read_csv('input/train.csv')
df_test = pd.read_csv('input/test.csv')

target_train = df_train['target'].values
id_test = df_test['id'].values

df_train=df_train.drop(['target','id'],axis=1)
df_test=df_test.drop(['id'], axis = 1)
combine= pd.concat([df_train,df_test],axis=0)






  

/home/rodrigo/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
# Performing rounding/binding
#combine1= combine

float_features = [a for a in combine.columns if combine[a].dtypes =='float64']

for column in float_features:
    combine[column] = round(combine[column],1)
    
#combine.head()
#combine1.head()


In [3]:



# Performing one hot encoding
cat_features = [a for a in combine.columns if a.endswith('cat')]
for column in cat_features:
    temp=pd.get_dummies(pd.Series(combine[column]))
    combine=pd.concat([combine,temp],axis=1)
    combine=combine.drop([column],axis=1)

df_train=combine[:df_train.shape[0]]
df_test=combine[df_train.shape[0]:]

train = np.array(df_train)
test = np.array(df_test)

print ("The train shape is:",train.shape)
print ('The test shape is:',test.shape)


# TODO: Import any additional functionality you may need here
from sklearn.model_selection import train_test_split

train_X, valid_X, train_y, valid_y = train_test_split(train, target_train, test_size=0.2, random_state=42)

xgb_preds = []


#K = 5
#kf = KFold(n_splits = K, random_state = 3228,shuffle=True)


#for train_index, test_index in kf.split(train):
#    train_X, valid_X = train[train_index], train[test_index]
#    train_y, valid_y = target_train[train_index], target_train[test_index]

    # params configuration also from the1owl's kernel
    # https://www.kaggle.com/the1owl/forza-baseline
xgb_params = {'eta': 0.02, 'max_depth': 5, 'subsample': 0.9, 'colsample_bytree': 0.9, 'objective': 'binary:logistic', 'eval_metric': 'auc', 'seed': 99, 'silent': True}

d_train = xgb.DMatrix(train_X, train_y)
d_valid = xgb.DMatrix(valid_X, valid_y)
d_test = xgb.DMatrix(test)
    
watchlist = [(d_train, 'train'), (d_valid, 'valid')]
model = xgb.train(xgb_params, d_train, 2000,  watchlist, feval=gini_xgb, maximize=True, verbose_eval=5, early_stopping_rounds=100)
                        
xgb_pred = model.predict(d_test)
xgb_preds.append(list(xgb_pred))
    
    
    
#preds=[]
#for i in range(len(xgb_preds[0])):
#    sum=0
#    for j in range(K):
#        sum+=xgb_preds[j][i]
#    preds.append(sum / K)

#output = pd.DataFrame({'id': id_test, 'target': preds})
#output.to_csv("{}-foldCV_avg_sub.csv".format(K), index=False)  

The train shape is: (595212, 227)
The test shape is: (892816, 227)
[0]	train-gini:0.213558	valid-gini:0.219502
Multiple eval metrics have been passed: 'valid-gini' will be used for early stopping.

Will train until valid-gini hasn't improved in 100 rounds.
[5]	train-gini:0.243139	valid-gini:0.243406
[10]	train-gini:0.244421	valid-gini:0.243173
[15]	train-gini:0.246818	valid-gini:0.244675
[20]	train-gini:0.249014	valid-gini:0.246123
[25]	train-gini:0.248846	valid-gini:0.24567
[30]	train-gini:0.250141	valid-gini:0.246227
[35]	train-gini:0.2518	valid-gini:0.24688
[40]	train-gini:0.252408	valid-gini:0.24705
[45]	train-gini:0.252903	valid-gini:0.24671
[50]	train-gini:0.253228	valid-gini:0.246675
[55]	train-gini:0.253632	valid-gini:0.246556
[60]	train-gini:0.254815	valid-gini:0.247479
[65]	train-gini:0.2559	valid-gini:0.247599
[70]	train-gini:0.256536	valid-gini:0.248137
[75]	train-gini:0.256457	valid-gini:0.247688
[80]	train-gini:0.257421	valid-gini:0.248281
[85]	train-gini:0.258962	valid-g

In [ ]:

trn_df = pd.read_csv("input/train.csv")
f_bins = [f for f in trn_df.columns if "_bin" in f]
f_cats = [f for f in trn_df.columns if "_cat" in f]
f_flts = [f for f in trn_df.columns if f not in f_bins + f_cats + ["target", "id"]]
f_all = [f_bins + f_cats + f_flts]



f_scores = pd.DataFrame(np.zeros((len(f_all), 2)), columns=["feature", "score"])
for i_f, f in enumerate(f_bins + f_cats + f_flts):
    # Get the score for each feature
    # If trn_df[f] has a negative score then - trn_df[f] has a positive one
    # Remember it's all about sorting !
    f_scores.loc[i_f] = [f, abs(gini_normalized(trn_df.target.values, trn_df[f].values))]
f_scores.sort_values(by="score", ascending=False, inplace=True)
f_scores.head(5)

In [4]:




submission = pd.read_csv('output/sample_submission.csv')

submission_output = submission

submission_output['target'] = xgb_pred


submission_output.to_csv('output/submission_output_xgb.csv',index=False)

print ("File generated!")


File generated!
